In [1]:
model_detection_path = '/media/prateek/prateek_space/intel_model_dump/intel/person-vehicle-bike-detection-crossroad-0078/FP16/person-vehicle-bike-detection-crossroad-0078.xml'
model_detection_path = '/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/intel/vehicle-detection-0201/FP16-INT8/vehicle-detection-0201.xml'

# cpu_extension = '../build_samples/intel64/Release/lib/libcpu_extension.so'
attr_model_path = "/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/intel/vehicle-attributes-recognition-barrier-0042/FP16-INT8/vehicle-attributes-recognition-barrier-0042.xml"

license_plate_detector_path = "/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/public/vehicle-license-plate-detection-barrier-0123/model/vehicle-license-plate-detection-barrier-0123.xml"

In [2]:
import openvino

In [3]:
import os
os.sys.path.append("../utilities/")

In [4]:
import time,cv2,importlib
import numpy as np
import IPython.display as Disp
import support_utility_openvino

In [5]:
support_utility_openvino = importlib.reload(support_utility_openvino)

In [6]:
attr_detection = support_utility_openvino.async_infer(4)
plate_detector = support_utility_openvino.async_infer(4)

Available Devices :  ['CPU', 'GNA', 'GPU']
Available Devices :  ['CPU', 'GNA', 'GPU']


In [7]:
vehicle_detection0 = support_utility_openvino.async_infer(4,ie_network=attr_detection.ie)

# vehicle_detection1 = support_utility_openvino.async_infer(4,ie_network=attr_detection.ie)

Available Devices :  ['CPU', 'GNA', 'GPU']


In [8]:
attr_detection.load_model(model_path=attr_model_path,device="MULTI:CPU,GPU")


model inputs : dict_keys(['input'])
model outputs :  dict_keys(['color', 'type'])


In [9]:
plate_detector.load_model(model_path=license_plate_detector_path,device="CPU")

OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['Placeholder'])
model outputs :  dict_keys(['DetectionOutput_'])


In [10]:
vehicle_detection0.load_model(model_path=model_detection_path,device="CPU")
output_support = support_utility_openvino.create_plot()

OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image'])
model outputs :  dict_keys(['detection_out'])


In [10]:
vehicle_detection1.load_model(model_path=model_detection_path1,device="CPU")
output_support = support_utility_openvino.create_plot()

OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'INT8', 'BIN']
model inputs : dict_keys(['data'])
model outputs :  dict_keys(['detection_out'])


In [11]:
vehicle_colors = ["White", "gray", "yellow", "red", "green", "blue", "black"]
vehicle_type = ["Car", "bus", "truck", "van"]

In [17]:
output_support.get_resolution_thresh(frame)

(0.07588541666666666, 0.7784953703703704)

In [12]:
channel = '/media/prateek/prateek_space/dataset/vehicle_related/china_number_plate_detection.mp4'
channel = '/media/prateek/prateek_space/dataset/veraval_test_video/vehicle/CSY_IP Camera9_CSY_20191022113859_20191022114100_7495885.mp4'
channel = '/media/prateek/prateek_space/dataset/veraval_test_video/vehicle/20191017_20191017095543_20191017100900_095957.mp4'
# channel = '/media/prateek/prateek_space/dataset/veraval_test_video/vehicle/Time office_IP Camera4_Time office_20190906150322_20190906150546_18994919.mp4'
channel = "/media/prateek/shared_space/DATA_N_MODELS/useful_vids/general/4K camera example for Traffic Monitoring (Road)-jjlBnrzSGjc.mp4"
channel = "/media/prateek/shared_space/DATA_N_MODELS/useful_vids/general/vehicle/indian_toll_gate.mp4"
cap = cv2.VideoCapture(channel)
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
ret,frame = cap.read()
fps = []
thresh = .4
res_range = [0,.9]
vehicle_detection = vehicle_detection0
vehicle_detection.reinit_model()
attr_detection.reinit_model()
write_video = False
if write_video:
    fourcc = cv2.VideoWriter_fourcc(*'h264')
    videowriter = cv2.VideoWriter("../output_vids/vehicle/veraval/"+"Tankfilling"+".mp4",fourcc, 20, (int(cap.get(3)),int(cap.get(4))))

while True:
    fps = fps
#     Disp.clear_output(wait=True)
    t1 = time.time()
    ret,frame = cap.read()
    if not ret:
        break
    
    vehicle_detection.predict(frame)
    if vehicle_detection.frame_processed>=vehicle_detection.num_requests:
        try:
            frame,attr,res = vehicle_detection.postprocess_op()
#             res[0] = np.expand_dims(res[0][res[0][:,:,:,1]==2],[0,1])
            op,bboxes,res_filt = output_support.trim_frame_with_result(frame,res[0],threshold=thresh,return_results = True,resolution_thresh_range = res_range)
            attrs = attr_detection.predict_batch_async(op)
            attrs = [(vehicle_colors[np.argmax(att[0])], vehicle_type[np.argmax(att[1])]) for att in attrs]
#             dets.append(len(attrs))
            fps.append(1/(time.time()-t1))
            op_frame = output_support.write_text(frame,"FPS = "+str(np.mean(fps,dtype='int')),text_color = (0,0,0),font_scale=1,font_thickness=2,highlight_color = (127,0,255))
            for idx,b in enumerate(bboxes):
                op_frame = output_support.create_bbox_with_text(op_frame,(tuple(b[:2]),tuple(b[2:])),",".join(attrs[idx]))
#             op_frame = output_support.label_obj_detection(frame,res[0],labels=['vehicle','person','car'],font_scale=1,font_thickness = 1,thresh=.5)
        except Exception as e:
            print(e)
#         print(vehicle_detection.cursor_id,vehicle_detection.frame_processed)
        cv2.imshow("preview",op_frame)
        k = cv2.waitKey(1)
        if k==27:
            break
        if write_video:
            videowriter.write(op_frame)
# cap.release()
if write_video:
    videowriter.release()
cap.release()
cv2.destroyAllWindows()
        
        
    

#### testing on images

In [12]:
file_path = "/media/prateek/prateek_space/dataset/veraval_production/4July/TankerFilling/VEHICLE/"
def get_all_file_path(source_dir,ext='.csv'):
    """
    all images with csv extension exist in set of dirs
    """
    op =[]
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith(ext):
                
                 op.append(os.path.join(os.path.abspath(root), file))
    return op
all_jpg = get_all_file_path(file_path,".jpg")

In [24]:


fps = []
thresh = .5
res_range = [0.05,.2]
vehicle_detection1.reinit_model()
attr_detection.reinit_model()
size = (1920,1080)
write_video = False
if write_video:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videowriter = cv2.VideoWriter("../output_vids/vehicle/veraval/"+"dispatch_liquid"+".mp4",fourcc, 20, size)

for f in all_jpg:
    cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
    fps = fps[-100:]
#     Disp.clear_output(wait=True)
    frame = cv2.imread(f)
    t1 = time.time()

#     if not ret:
#         break
    
    vehicle_detection1.predict(frame)
    if vehicle_detection1.frame_processed>=vehicle_detection1.num_requests:
        try:
            frame,attr,res = vehicle_detection1.postprocess_op()
#             res[0] = np.expand_dims(res[0][res[0][:,:,:,1]==2],[0,1])
            op,bboxes,res_filt = output_support.trim_frame_with_result(frame,res[0],threshold=thresh,return_results = True)#,resolution_thresh_range=res_range)
            attrs = attr_detection.predict_batch_async(op)
            attrs = [(vehicle_colors[np.argmax(att[0])], vehicle_type[np.argmax(att[1])]) for att in attrs]
#             dets.append(len(attrs))
            fps.append(1/(time.time()-t1))
            op_frame = output_support.write_text(frame,"FPS = "+str(np.mean(fps,dtype='int')),text_color = (0,0,0),font_scale=1,font_thickness=2,highlight_color = (127,0,255))
            op_frame = output_support.write_text(op_frame,"number of detections = %s"%len(bboxes),location="top-right",text_color = (0,0,0),font_scale=1,font_thickness=2,highlight_color = (127,0,255))

            for idx,b in enumerate(bboxes):
                op_frame = output_support.create_bbox_with_text(op_frame,(tuple(b[:2]),tuple(b[2:])),",".join(attrs[idx]))
#             op_frame = output_support.label_obj_detection(frame,res[0],labels=['vehicle','person','car'],font_scale=1,font_thickness = 1,thresh=.5)
        except Exception as e:
            print(e)
#         print(vehicle_detection.cursor_id,vehicle_detection.frame_processed)
        cv2.imshow("preview",op_frame)
        k = cv2.waitKey()
        if k==ord('q'):
            break
            
            
        if write_video:
            videowriter.write(cv2.resize(op_frame,size))
# cap.release()
if write_video:
    videowriter.release()
# cap.release()
cv2.destroyAllWindows()

        
        
    

In [49]:
frame,attr,res = vehicle_detection.postprocess_op()

In [20]:
"number of detections = %s"%len(bboxes)

'number of detections = 0'

In [48]:
frame = cv2.imread(f)

t1 = time.time()
vehicle_detection1.predict(frame)